In [ ]:
from operator import itemgetter
from typing import Dict, TypedDict
from bs4 import BeautifulSoup as Soup
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_community.document_loaders.recursive_url_loader import (
    RecursiveUrlLoader,
)
from langgraph.graph import END, StateGraph

In [ ]:
load_dotenv("../.env")

In [ ]:
url = "https://python.langchain.com/docs/expression_language/"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))

concatenated_content = "\n\n\n --- \n\n\n".join(
    [d.page_content for d in d_reversed]
)

In [ ]:
class Code(BaseModel):
    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")


class GraphState(TypedDict):
    keys: Dict[str, any]

In [ ]:
model = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)
code_tool_ai = convert_to_openai_tool(Code)

llm_with_tool = model.bind(
    tools=[code_tool_ai],
    tool_choice={"type": "function", "function": {"name": "Code"}},
)

parser_tool = PydanticToolsParser(tools=[Code])

In [ ]:
template = """You are a coding assistant with expertise in LCEL, LangChain \
expression language. \n
Here is a full set of LCEL documentation:

<lcel_docs>\n
{context}
</lcel_docs>\n

Answer the user question based on the above provided documentation. \n
Ensure any code you provided can be executed with description of the code \
solution. \n
Then list the imports. And finally list the functioning code block. \n
Here is the user question: \n

<question>
{question}
</question>
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
)

template_addendum = """\n --- --- ---\n You previously tried to solve this \
problem. \n Here is your solution: \n\n

<code>
{generation}
</code>\n
\n--- --- ---\n
Here is the resulting error from code execution: \n\n

<error>
{error}
</error>\n
\n--- --- ---\n

Please re-try to answer this. \n
And finnaly list the functioning code block. Structure your answer with a \
description of the code solution. \n
Then list the imports. And finally list the functioning code block.\n
Here is the user question: \n

<question>
{question}
</question> \
"""

In [ ]:
chain = (
    {
        "context": lambda x: concatenated_content,
        "question": itemgetter("question"),
    }
    | prompt
    | llm_with_tool
    | parser_tool
)

In [ ]:
# chain.invoke({"question": "How to create a RAG chain in LCEL?"})

In [ ]:
def generate(state):
    state_dict = state["keys"]
    question = state_dict["question"]
    iter = state_dict["iterations"]

    if "error" in state_dict:
        print("ERROR: DO IT AGAIN")
        error = state_dict["error"]

        _template = template + template_addendum
        prompt = PromptTemplate(
            template=_template,
            input_variables=["context", "question", "error", "generation"],
        )

        chain = (
            {
                "context": lambda x: concatenated_content,
                "question": itemgetter("question"),
                "generation": itemgetter("generation"),
                "error": itemgetter("error"),
            }
            | prompt
            | llm_with_tool
            | parser_tool
        )
        code_solution = chain.invoke(
            {
                "question": question,
                "generation": str(code_solution[0]),
                "error": error,
            }
        )
    else:
        print("GENERATE SOLUTION")
        prompt = PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        )

        chain = (
            {
                "context": lambda x: concatenated_content,
                "question": itemgetter("question"),
            }
            | prompt
            | llm_with_tool
            | parser_tool
        )

        code_solution = chain.invoke({"question": question})
    iter = iter + 1
    return {
        "keys": {
            "generation": code_solution,
            "question": question,
            "iterations": iter,
        }
    }

In [ ]:
def check_code_imports(state):
    print("CHECK CODE IMPORTS")
    state_dict = state["keys"]
    question = state_dict["question"]
    code_solution = state_dict["generation"]
    imports = code_solution[0]["imports"]
    iter = state_dict["iterations"]

    try:
        exec(imports)
    except Exception as e:
        print("CODE IMPORT CHECK FAILED")
        error = f"Execution error: {e}"
        if "error" in state_dict:
            error_prev_runs = state_dict["error"]
            error = error_prev_runs + "\n --- Most Recent Error ---\n" + error
    else:
        print("CODE IMPORT SUCCESS")
        error = "None"
    return {
        "keys": {
            "generation": code_solution,
            "question": question,
            "error": error,
            "iterations": iter,
        }
    }

In [ ]:
def check_code_execution(state):
    print("--- CHECK CODE EXECUTION ---")
    state_dict = state["keys"]
    question = state_dict["question"]
    code_solution = state_dict["generation"]
    prefix = code_solution[0]["prefix"]
    imports = code_solution[0]["imports"]
    code = code_solution[0]["code"]
    code_block = imports + "\n\n" + code
    iter = state_dict["iterations"]

    try:
        exec(code_block)
    except Exception as e:
        print("CODE EXECUTION FAILED")
        if "error" in state_dict:
            error_prev_runs = state_dict["error"]
            error = error_prev_runs + "\n --- Most Recent Error ---\n" + str(e)
    else:
        print("CODE BLOCK CHECK: SUCCESS")
        error = "None"
    return {
        "keys": {
            "generation": code_solution,
            "question": question,
            "error": error,
            "iterations": iter,
            "prefix": prefix,
            "imports": imports,
            "code": code,
        }
    }

In [ ]:
def decide_to_check_code_exec(state):
    print("DECIDE TO TEST CODE EXECUTION")
    state_dict = state["keys"]
    question = state_dict["question"]
    code_solution = state_dict["generation"]
    error = state_dict["error"]

    if error == "None":
        print("DECISION: TEST CODE EXECUTION")
        return "check_code_execution"
    else:
        print("DECISION: RE-TRY SOLUTION")
        return "generate"

In [ ]:
def decide_to_finish(state):
    print("DECIDE TO FINISH")
    state_dict = state["keys"]
    question = state_dict["question"]
    error = state_dict["error"]

    if error == "None" or iter == 3:
        print("DECISION: TEST CODE EXECUTION")
        return "end"
    else:
        print("DECISION: RE-TRY SOLUTION")
        return "generate"

In [ ]:
workflow = StateGraph(GraphState)

workflow.add_node("generate", generate)
workflow.add_node("check_code_imports", check_code_imports)
workflow.add_node("check_code_execution", check_code_execution)

workflow.set_entry_point("generate")
workflow.add_edge("generate", "check_code_imports")
workflow.add_conditional_edges(
    "check_code_imports",
    decide_to_check_code_exec,
    {"check_code_execution": "check_code_execution", "generate": "generate"},
)
workflow.add_conditional_edges(
    "check_code_execution",
    decide_to_finish,
    {"end": END, "generate": "generate"},
)

app = workflow.compile()

In [ ]:
question = "I'm passing text key 'foo' to my prompt and want to process it with a function, process_text(...), prior to the prompt."
config = {"recursion_limit": 50}
answer = app.invoke(
    {"keys": {"question": question, "iterations": 0}}, config=config
)